In [1]:
import os
import sys

sys.path.append('/home/ubuntu/project')
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cv2
import math

from utils.video_pipeline import get_videos_from_file

from scenedetect.platform import tqdm
# Standard PySceneDetect imports:
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager, FrameTimecode
# For caching detection metrics and saving/loading to a stats file
from scenedetect.stats_manager import StatsManager

# For content-aware scene detection:
from scenedetect.detectors.content_detector import ContentDetector


def find_scenes(video_path):
    # type: (str) -> List[Tuple[FrameTimecode, FrameTimecode]]
    video_manager = VideoManager([video_path])
    stats_manager = StatsManager()
    # Construct our SceneManager and pass it our StatsManager.
    scene_manager = SceneManager(stats_manager)

    # Add ContentDetector algorithm (each detector's constructor
    # takes detector options, e.g. threshold).
    scene_manager.add_detector(ContentDetector())
    base_timecode = video_manager.get_base_timecode()


    scene_list = []

    try:
        # Set downscale factor to improve processing speed.
        video_manager.set_downscale_factor()

        # Start video_manager.
        video_manager.start()

        # Perform scene detection on video_manager.
        scene_manager.detect_scenes(frame_source=video_manager)

        # Obtain list of detected scenes.
        scene_list = scene_manager.get_scene_list(base_timecode)
        # Each scene is a tuple of (start, end) FrameTimecodes.

        frames = []

        for i, scene in enumerate(scene_list):
            frames.append(scene[1].get_frames())

    finally:
        video_manager.release()

    return frames

In [7]:
#frames = find_scenes('../tmp/tmp_720p.mp4')

video1_frames = get_videos_from_file('../tmp/tmp_720p.mp4')

frames = scene_detect(video1_frames)

H, W, _ = video1_frames[0].shape
    
dim = (W, H)

fourcc = cv2.VideoWriter_fourcc(*'XVID')   
video_tracked = cv2.VideoWriter(f'/home/ubuntu/project/tmp/scene_detect.mp4', fourcc, 20.0, dim)

i = 0

for f, frame in enumerate(video1_frames):
    if f > frames[i]:
        i += 1

    frame_draw = cv2.putText(frame, f'Scene_{i}', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0,0), 2)

    video_tracked.write(cv2.cvtColor(frame_draw, cv2.COLOR_RGB2BGR))
    
video_tracked.release()


:33, 57.65frames/s]
100%|██████████| 1929/1929 [00:52<00:00, 53.01frames/s]